## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI:

```bash
az login
```

# 💬 Chat Middleware

## Industry Use Case: Customer Service Message Filtering

This notebook demonstrates **chat middleware** for intercepting and modifying messages.

| Feature | FSI Application |
|---------|-----------------|
| **Message Observation** | Audit logging for compliance |
| **Message Modification** | PII redaction |
| **Response Override** | Block sensitive queries |

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv('../../.env', override=True)

print(f"✅ Environment loaded")

In [ ]:
from collections.abc import Awaitable, Callable

from agent_framework import (
    ChatContext,
    ChatMessage,
    ChatMiddleware,
    ChatResponse,
    Role,
    chat_middleware,
)
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

print("✅ All imports loaded")

## Define FSI Tool Function

In [ ]:
def get_account_balance(account_id: str) -> str:
    """Get account balance for a customer."""
    return f"Account {account_id} has a balance of $15,432.50"

print("✅ Tool defined: get_account_balance")

## Define Chat Middleware

Chat middleware intercepts requests before they reach the AI service.

In [ ]:
# Class-based middleware for message observation
class InputObserverMiddleware(ChatMiddleware):
    """Observes and logs input messages."""

    async def process(
        self,
        context: ChatContext,
        next: Callable[[ChatContext], Awaitable[None]],
    ) -> None:
        print("[Observer] Observing input messages:")
        for i, message in enumerate(context.messages):
            content = message.text if message.text else str(message.contents)
            print(f"  Message {i + 1} ({message.role.value}): {content[:50]}...")
        
        await next(context)
        print("[Observer] Processing completed")


# Function-based middleware for security filtering
@chat_middleware
async def security_middleware(
    context: ChatContext,
    next: Callable[[ChatContext], Awaitable[None]],
) -> None:
    """Blocks sensitive information requests."""
    blocked_terms = ["password", "secret", "ssn", "social security"]

    for message in context.messages:
        if message.text:
            for term in blocked_terms:
                if term in message.text.lower():
                    print(f"[Security] BLOCKED: Found '{term}'")
                    context.result = ChatResponse(
                        messages=[ChatMessage(role=Role.ASSISTANT, text="I cannot process requests for sensitive information.")]
                    )
                    context.terminate = True
                    return

    await next(context)

print("✅ Middleware defined: InputObserverMiddleware, security_middleware")

## Example 1: Class-based Chat Middleware

In [ ]:
async def class_based_example():
    print("=== Class-based Chat Middleware ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(credential=credential).as_agent(
            name="ObserverAgent",
            instructions="You are a helpful banking assistant.",
            middleware=[InputObserverMiddleware()],
            tools=get_account_balance,
        ) as agent,
    ):
        query = "What is the balance for account ACC-123?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result.text if result.text else 'No response'}")

await class_based_example()

## Example 2: Function-based Security Middleware

In [ ]:
async def security_example():
    print("=== Security Middleware ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(credential=credential).as_agent(
            name="SecureAgent",
            instructions="You are a helpful assistant.",
            middleware=[security_middleware],
        ) as agent,
    ):
        # Normal query
        print("\n--- Normal Query ---")
        query = "Hello, how are you?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result.text if result.text else 'No response'}")

        # Blocked query
        print("\n--- Blocked Query ---")
        query = "What is my password?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result.text if result.text else 'No response'}")

await security_example()

## Example 3: Run-level Middleware

In [ ]:
async def run_level_example():
    print("=== Run-level Middleware ===")
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(credential=credential).as_agent(
            name="FlexibleAgent",
            instructions="You are a helpful assistant.",
            # No middleware at agent level
        ) as agent,
    ):
        # Run without middleware
        print("\n--- Without Middleware ---")
        query = "Hello"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Agent: {result.text if result.text else 'No response'}")

        # Run with middleware for this call only
        print("\n--- With Run-level Middleware ---")
        query = "What is my SSN?"
        print(f"User: {query}")
        result = await agent.run(query, middleware=[security_middleware])
        print(f"Agent: {result.text if result.text else 'No response'}")

await run_level_example()

## 📋 Key Takeaways

| Feature | Description |
|---------|-------------|
| **ChatMiddleware class** | Class-based middleware with `process()` method |
| **@chat_middleware** | Function decorator for chat middleware |
| **Agent-level** | Middleware applies to all runs |
| **Run-level** | Middleware applies to specific run only |
| **context.terminate** | Stop execution and return override response |